<a href="https://colab.research.google.com/github/LucioJuniorMachado/Trabalho_Redes_Neurais/blob/main/covid_19_(5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Vamos Classificar RX de covid e normais

In [ ]:
!pip install tensorflow==2.8.0

In [ ]:
!pip install protobuf==3.20

In [ ]:
import sys
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os
import glob
from imutils import paths

Vamos a procurar o caminho de nossa base

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
workdir_path = '/content/drive/My Drive/dataset-20250408T150050Z-001'  # Inserir o local da pasta onde estão os arquivos de entrada (treino e teste)
os.chdir(workdir_path)

#Vamos colocar os parametros de nosso modelo

In [ ]:
INIT_LR = 1e-3
EPOCHS = 15 # PODE MUDAR
BS = 8

# toma la lista de imágenes en nuestro directorio de conjuntos de datos, luego inicializa
# la lista de datos (es decir, imágenes) e imágenes de clase

In [ ]:

dataset_path = 'dataset'
print("[INFO] loading images...")
imagePaths = list(paths.list_images(dataset_path))
data = []
labels = []

In [ ]:
# recorrer las rutas de la imagen
for imagePath in imagePaths:
    # extrae la etiqueta de clase del nombre del archivo
    label = imagePath.split(os.path.sep)[-2]

   # cargue la imagen, intercambie canales de color y cambie su tamaño para que sea una
   # 224x224 píxeles sin tener en cuenta la relación de aspecto
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))

    # actualizar las listas de datos y etiquetas, respectivamente
    data.append(image)
    labels.append(label)
print("labels: ", np.unique(labels))

In [ ]:
# convierta los datos y las etiquetas en matrices NumPy mientras escala el píxel
# intensidades en el rango [0, 255]
data = np.array(data) / 255.0
labels = np.array(labels)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(data[labels=='covid'][1])
plt.show()

In [ ]:
# realizar una codificación one-hot en las etiquetas
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [ ]:
# particione los datos en divisiones de entrenamiento y prueba usando el 80% de
# los datos para entrenamiento y el 20% restante para pruebas
(trainX, testX, trainY, testY) = train_test_split(data, labels,
                                                  test_size=0.20, stratify=labels, random_state=42)

## Data Augmentation

In [ ]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
    rotation_range=40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

In [ ]:
# cargue la red VGG16, asegurándose de que queden los conjuntos de capas FC principales
# off
baseModel = VGG16(weights="imagenet", include_top=False,
                  input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
#headModel = MaxPooling2D(pool_size=(2, 2))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.3)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False

# compile our model
print("[INFO] compiling model...")
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)  # Replace 'decay' with 'learning_rate'
model.compile(loss="binary_crossentropy", optimizer=opt,
              metrics=["accuracy"])

In [ ]:
baseModel = VGG16(weights="imagenet", include_top=False,
                  input_tensor=Input(shape=(224, 224, 3)))

f1 = baseModel.layers[1].output
f2 = baseModel.layers[2].output
f3 = baseModel.layers[4].output
f4 = baseModel.layers[5].output
feature_maps = Model(inputs=baseModel.input, outputs=[f1, f2, f3, f4])

In [ ]:
y = np.argmax(trainY, axis=-1)

In [ ]:
feat1, feat2, feat3, feat4 = feature_maps.predict(trainX[y==1][0:1])

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
axs[0, 0].imshow(feat1[0, :, :, 0:3])
axs[0, 1].imshow(feat2[0, :, :, 6:9])
axs[1, 0].imshow(feat3[0, :, :, 0:3])
axs[1, 1].imshow(feat4[0, :, :, 3:6])
# plt.subplots
# plt.imshow(feat1[0, :, :, 0:3])
plt.show()

##passo os dados de entrada

In [ ]:
# train the head of the network
print("[INFO] training head...")
H = model.fit(
    trainAug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)



In [ ]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
                            target_names=lb.classes_))

# compute the confusion matrix and and use it to derive the raw
# accuracy, sensitivity, and specificity
cm = confusion_matrix(testY.argmax(axis=1), predIdxs)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])

# show the confusion matrix, accuracy, sensitivity, and specificity
print(cm)
print("acc: {:.4f}".format(acc))

# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()
# plt.savefig(args["plot"])

# serialize the model to disk
# print("[INFO] saving COVID-19 detector model...")
# model.save(args["model"], save_format="h5")

# Implementando o modelo vgg19

In [ ]:
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications.vgg19 import VGG19 # Import VGG19 model
from tensorflow.keras.applications.vgg19 import decode_predictions
from tensorflow.keras.preprocessing.image import img_to_array # Assuming this was imported earlier
from numpy import expand_dims # Assuming this was imported earlier
import cv2 # Assuming this was imported earlier


# The 'image' variable is already a NumPy array from previous steps.
# If you intend to process a *new* image, you would load it here.
# For demonstration, let's assume 'image' is the NumPy array you want to use.

# If the image is not already 224x224, resize it using cv2
# Check if the image needs resizing (assuming HxWxC format)
if image.shape[0] != 224 or image.shape[1] != 224:
    # Resize the image using OpenCV
    image = cv2.resize(image, (224, 224))

# The image is already a NumPy array, so img_to_array might not be necessary
# depending on how 'image' was obtained. If 'image' is guaranteed to be a NumPy array
# with the correct shape and dtype after resizing (if needed), this line might be redundant.
# However, if 'image' was a PIL Image object *before* the error, and you want to
# convert it to a NumPy array, this line would be correct *after* the PIL resize.
# Since the error indicates 'image' is already a NumPy array, we'll assume it is.
# If you were starting with a PIL Image, the workflow would be:
# from PIL import Image
# image_pil = Image.open("your_image.jpg")
# image_pil = image_pil.resize((224, 224))
# image_array = img_to_array(image_pil)
# Or more simply using cv2:
# image_cv2 = cv2.imread("your_image.jpg")
# image_array = cv2.resize(image_cv2, (224, 224)) # cv2 reads as BGR, so need to convert if model expects RGB

# Let's assume 'image' is the correctly sized (224, 224, 3) NumPy array in RGB format.

# Expanda a dimensão para criar um batch de tamanho 1 (o modelo espera um batch)
image_array = expand_dims(image, axis=0) # Use the existing 'image' array

# Pré-processe a imagem para o VGG19
# Isso subtrai a média dos canais VGG19 e garante a ordem correta dos canais (RGB para BGR se necessário)
# Note: preprocess_input for VGG models typically converts RGB to BGR and subtracts mean.
image_array_processed = preprocess_input(image_array)

# Carregue o modelo VGG19 pré-treinado no ImageNet
# Fix: Instantiate the VGG19 model correctly
model_vgg19 = VGG19(weights="imagenet", include_top=True)

# Realize a inferência na imagem pré-processada
predictions = model_vgg19.predict(image_array_processed)

# O modelo VGG19 com include_top=True retorna probabilidades para as 1000 classes do ImageNet.
# Para interpretar os resultados, podemos usar a função decode_predictions do Keras.

# Decodifique as 3 principais previsões
decoded_predictions = decode_predictions(predictions, top=3)[0]

print("\n--- Resultados da Inferência VGG19 ---")
for class_id, class_name, score in decoded_predictions:
    print(f"Classe: {class_name}, Probabilidade: {score:.4f}")

# Treinamento da rede vgg19 com 20 épocas

In [ ]:
# Load the VGG19 network, ensuring the head FC layers are left off
# Change VGG16 to VGG19
baseModel = VGG19(weights="imagenet", include_top=False,
                  input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.3)(headModel)
# Assuming binary classification (covid/normal), output layer has 2 units
headModel = Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False

# compile our model
print("[INFO] compiling model...")
# Use the optimizer defined previously
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
# Use binary_crossentropy for binary classification
model.compile(loss="binary_crossentropy", optimizer=opt,
              metrics=["accuracy"])

# Train the head of the network
print("[INFO] training head...")
# Change EPOCHS to 20
EPOCHS = 20
H = model.fit(
    trainAug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

# The code for evaluation and plotting already exists and will use the results from this training run.
# No need to repeat the evaluation and plotting code.

# Comparação de acurácia e perda de validação entre as redes vgg16 e vgg19

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# Save the training history for VGG16
history_vgg16 = H

# Now, redefine and train the VGG19 model
print("\n[INFO] Training VGG19 model...")

# Load the VGG19 network, ensuring the head FC layers are left off
baseModel_vgg19 = VGG19(weights="imagenet", include_top=False,
                        input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel_vgg19 = baseModel_vgg19.output
headModel_vgg19 = Flatten(name="flatten_vgg19")(headModel_vgg19)
headModel_vgg19 = Dense(128, activation="relu")(headModel_vgg19)
headModel_vgg19 = Dropout(0.3)(headModel_vgg19)
headModel_vgg19 = Dense(2, activation="softmax")(headModel_vgg19)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model_vgg19 = Model(inputs=baseModel_vgg19.input, outputs=headModel_vgg19)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel_vgg19.layers:
    layer.trainable = False

# compile our model
print("[INFO] compiling VGG19 model...")
# Use the optimizer defined previously
opt_vgg19 = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model_vgg19.compile(loss="binary_crossentropy", optimizer=opt_vgg19,
                    metrics=["accuracy"])

# Train the head of the network
print("[INFO] training VGG19 head...")
# Use the same number of EPOCHS for fair comparison, or define a new one if needed
EPOCHS_VGG19 = EPOCHS # Using the same EPOCHS value
history_vgg19 = model_vgg19.fit(
    trainAug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS_VGG19)

# --- Compare and Plot Results ---

# Plot training loss and accuracy for both models
N_vgg16 = len(history_vgg16.history["loss"])
N_vgg19 = len(history_vgg19.history["loss"])

plt.style.use("ggplot")
plt.figure(figsize=(12, 6))

# Plot training loss
plt.subplot(1, 2, 1)
plt.plot(np.arange(0, N_vgg16), history_vgg16.history["loss"], label="VGG16 train_loss")
plt.plot(np.arange(0, N_vgg16), history_vgg16.history["val_loss"], label="VGG16 val_loss")
plt.plot(np.arange(0, N_vgg19), history_vgg19.history["loss"], label="VGG19 train_loss")
plt.plot(np.arange(0, N_vgg19), history_vgg19.history["val_loss"], label="VGG19 val_loss")
plt.title("Training and Validation Loss Comparison")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="upper right")

# Plot training accuracy
plt.subplot(1, 2, 2)
plt.plot(np.arange(0, N_vgg16), history_vgg16.history["accuracy"], label="VGG16 train_acc")
plt.plot(np.arange(0, N_vgg16), history_vgg16.history["val_accuracy"], label="VGG16 val_acc")
plt.plot(np.arange(0, N_vgg19), history_vgg19.history["accuracy"], label="VGG19 train_acc")
plt.plot(np.arange(0, N_vgg19), history_vgg19.history["val_accuracy"], label="VGG19 val_acc")
plt.title("Training and Validation Accuracy Comparison")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower right")

plt.tight_layout()
plt.show()

# Evaluate VGG19 model and print classification report and confusion matrix
print("\n[INFO] Evaluating VGG19 network...")
predIdxs_vgg19 = model_vgg19.predict(testX, batch_size=BS)
predIdxs_vgg19 = np.argmax(predIdxs_vgg19, axis=1)

print("\n--- Classification Report VGG19 ---")
print(classification_report(testY.argmax(axis=1), predIdxs_vgg19,
                            target_names=lb.classes_))

cm_vgg19 = confusion_matrix(testY.argmax(axis=1), predIdxs_vgg19)
total_vgg19 = sum(sum(cm_vgg19))
acc_vgg19 = (cm_vgg19[0, 0] + cm_vgg19[1, 1]) / total_vgg19

print("\n--- Confusion Matrix VGG19 ---")
print(cm_vgg19)
print("VGG19 acc: {:.4f}".format(acc_vgg19))
